# Tutorial 15: FastMCP Server Basics

## Building Custom MCP Servers for Enterprise AI Agents

### What You'll Learn

In this tutorial, you'll learn how to build a **FastMCP server** that exposes custom tools to AI agents using the Model Context Protocol (MCP). This is the foundation for building enterprise-grade, scalable AI agent architectures.

**Key Concepts:**
- Model Context Protocol (MCP) overview
- Creating a FastMCP server
- Defining custom tools with schemas
- Testing MCP servers locally
- Integrating MCP servers with AI agents

**Use Case:** Building a Travel Booking MCP Server with flight search, hotel availability, and currency conversion tools.

**Duration:** 45 minutes

---

### Prerequisites

- Completed tutorials 01-14
- Python 3.11+
- Understanding of REST APIs
- Azure OpenAI or OpenAI API access

## Part 1: Understanding Model Context Protocol (MCP)

### What is MCP?

**Model Context Protocol (MCP)** is an open standard for connecting AI assistants to external data sources and tools. Think of it as a standardized way for AI agents to:

- Access external data (databases, APIs, file systems)
- Execute functions and tools
- Maintain context across interactions
- Scale agent capabilities without modifying agent code

### Why FastMCP?

**FastMCP** is a Python framework that makes it easy to build MCP servers with:
- Simple, decorator-based tool definitions
- Automatic schema generation
- Built-in validation
- Easy local testing
- Production-ready server implementation

### Architecture

```
┌─────────────────┐
│   AI Agent      │
│  (OpenAI/Azure) │
└────────┬────────┘
         │ MCP Protocol
         │
┌────────┴────────┐
│  FastMCP Server │
│  ┌───────────┐  │
│  │  Tool 1   │  │
│  │  Tool 2   │  │
│  │  Tool 3   │  │
│  └───────────┘  │
└─────────────────┘
```

## Part 2: Setup and Installation

First, let's install the required packages for building our FastMCP server.

In [2]:
# Install FastMCP and required dependencies
# Using latest compatible versions
%pip install fastmcp uvicorn python-dotenv

  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
Using cached referencing-0.36.2-py3-none-any.whl (26 kB)
Using cached referencing-0.36.2-py3-none-any.whl (26 kB)
  Attempting uninstall: referencing
    Found existing installation: referencing 0.37.0
    Uninstalling referencing-0.37.0:
      Successfully uninstalled referencing-0.37.0

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import required libraries
import os
from datetime import datetime, timedelta
from typing import Optional, List
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import json

# Load environment variables
load_dotenv()

print("Environment loaded successfully")

Environment loaded successfully


## Part 3: Creating Your First FastMCP Server

Let's build a Travel Booking MCP Server with three tools:
1. **search_flights** - Search for available flights
2. **check_hotel_availability** - Check hotel room availability
3. **convert_currency** - Convert between currencies

We'll create a file called `travel_mcp_server.py` that contains our MCP server implementation.

In [4]:
# Create the MCP server file
server_code = '''
"""
Travel Booking MCP Server
A FastMCP server that provides travel-related tools for AI agents
"""

from fastmcp import FastMCP
from datetime import datetime, timedelta
from typing import List, Optional, Dict
from pydantic import BaseModel, Field
import random

# Initialize the MCP server
mcp = FastMCP("Travel Booking Server")

# Data models for structured responses
class Flight(BaseModel):
    """Flight information"""
    flight_number: str
    airline: str
    departure: str
    arrival: str
    departure_time: str
    arrival_time: str
    price: float
    currency: str = "USD"
    available_seats: int

class Hotel(BaseModel):
    """Hotel information"""
    hotel_name: str
    location: str
    room_type: str
    price_per_night: float
    currency: str = "USD"
    available_rooms: int
    amenities: List[str]

# Underlying functions that return JSON-serializable data
def _search_flights(
    origin: str,
    destination: str,
    departure_date: str,
    passengers: int = 1
) -> List[Dict]:
    """
    Search for available flights between two cities.
    
    Args:
        origin: Departure city (e.g., "New York", "NYC")
        destination: Arrival city (e.g., "London", "LHR")
        departure_date: Date in YYYY-MM-DD format
        passengers: Number of passengers (default: 1)
    
    Returns:
        List of available flights with details (as dicts)
    """
    # Simulate flight search (in production, call real APIs)
    airlines = ["Delta", "United", "American", "British Airways"]
    
    flights = []
    for i in range(3):
        dep_time = datetime.strptime(departure_date, "%Y-%m-%d")
        dep_time = dep_time.replace(hour=random.randint(6, 20), minute=random.choice([0, 15, 30, 45]))
        arr_time = dep_time + timedelta(hours=random.randint(6, 12))
        
        flight = {
            "flight_number": f"{random.choice(['DL', 'UA', 'AA', 'BA'])}{random.randint(100, 999)}",
            "airline": random.choice(airlines),
            "departure": origin,
            "arrival": destination,
            "departure_time": dep_time.strftime("%Y-%m-%d %H:%M"),
            "arrival_time": arr_time.strftime("%Y-%m-%d %H:%M"),
            "price": round(random.uniform(300, 1200), 2),
            "currency": "USD",
            "available_seats": random.randint(5, 50)
        }
        flights.append(flight)
    
    return flights

def _check_hotel_availability(
    location: str,
    check_in: str,
    check_out: str,
    guests: int = 2
) -> List[Dict]:
    """
    Check hotel room availability in a specific location.
    
    Args:
        location: City or area name
        check_in: Check-in date in YYYY-MM-DD format
        check_out: Check-out date in YYYY-MM-DD format
        guests: Number of guests (default: 2)
    
    Returns:
        List of available hotels with details (as dicts)
    """
    # Simulate hotel search (in production, call real APIs)
    hotel_names = ["Grand Plaza Hotel", "Seaside Resort", "Downtown Suites", "Airport Inn"]
    room_types = ["Standard Room", "Deluxe Room", "Suite", "Executive Suite"]
    
    hotels = []
    for i in range(4):
        hotel = {
            "hotel_name": random.choice(hotel_names),
            "location": location,
            "room_type": random.choice(room_types),
            "price_per_night": round(random.uniform(80, 350), 2),
            "currency": "USD",
            "available_rooms": random.randint(1, 10),
            "amenities": random.sample(["WiFi", "Pool", "Gym", "Restaurant", "Spa", "Parking"], k=3)
        }
        hotels.append(hotel)
    
    return hotels

def _convert_currency(
    amount: float,
    from_currency: str,
    to_currency: str
) -> Dict:
    """
    Convert amount from one currency to another.
    
    Args:
        amount: Amount to convert
        from_currency: Source currency code (e.g., "USD", "EUR", "GBP")
        to_currency: Target currency code
    
    Returns:
        Dictionary with conversion details
    """
    # Simulate currency conversion (in production, call real exchange rate API)
    exchange_rates = {
        "USD": {"EUR": 0.85, "GBP": 0.73, "JPY": 110.0, "USD": 1.0},
        "EUR": {"USD": 1.18, "GBP": 0.86, "JPY": 129.0, "EUR": 1.0},
        "GBP": {"USD": 1.37, "EUR": 1.16, "JPY": 150.0, "GBP": 1.0},
        "JPY": {"USD": 0.0091, "EUR": 0.0078, "GBP": 0.0067, "JPY": 1.0}
    }
    
    rate = exchange_rates.get(from_currency, {}).get(to_currency, 1.0)
    converted_amount = round(amount * rate, 2)
    
    return {
        "original_amount": amount,
        "original_currency": from_currency,
        "converted_amount": converted_amount,
        "converted_currency": to_currency,
        "exchange_rate": rate,
        "timestamp": datetime.now().isoformat()
    }

def _get_server_info() -> Dict:
    """Get information about this MCP server"""
    return {
        "name": "Travel Booking Server",
        "version": "1.0.0",
        "tools": ["search_flights", "check_hotel_availability", "convert_currency"],
        "description": "MCP server for travel booking operations"
    }

# MCP tool wrappers - convert results to Pydantic models for MCP
@mcp.tool()
def search_flights(
    origin: str,
    destination: str,
    departure_date: str,
    passengers: int = 1
) -> List[Flight]:
    """
    Search for available flights between two cities.
    
    Args:
        origin: Departure city (e.g., "New York", "NYC")
        destination: Arrival city (e.g., "London", "LHR")
        departure_date: Date in YYYY-MM-DD format
        passengers: Number of passengers (default: 1)
    
    Returns:
        List of available flights with details
    """
    flights_data = _search_flights(origin, destination, departure_date, passengers)
    return [Flight(**f) for f in flights_data]

@mcp.tool()
def check_hotel_availability(
    location: str,
    check_in: str,
    check_out: str,
    guests: int = 2
) -> List[Hotel]:
    """
    Check hotel room availability in a specific location.
    
    Args:
        location: City or area name
        check_in: Check-in date in YYYY-MM-DD format
        check_out: Check-out date in YYYY-MM-DD format
        guests: Number of guests (default: 2)
    
    Returns:
        List of available hotels with details
    """
    hotels_data = _check_hotel_availability(location, check_in, check_out, guests)
    return [Hotel(**h) for h in hotels_data]

@mcp.tool()
def convert_currency(
    amount: float,
    from_currency: str,
    to_currency: str
) -> dict:
    """
    Convert amount from one currency to another.
    
    Args:
        amount: Amount to convert
        from_currency: Source currency code (e.g., "USD", "EUR", "GBP")
        to_currency: Target currency code
    
    Returns:
        Dictionary with conversion details
    """
    return _convert_currency(amount, from_currency, to_currency)

@mcp.tool()
def get_server_info() -> dict:
    """Get information about this MCP server"""
    return _get_server_info()

# ASGI app for uvicorn (used in container deployment - Tutorial 16)
app = mcp.http_app()

if __name__ == "__main__":
    # Run the server locally via stdio transport
    mcp.run()
'''

# Write the server code to a file
with open('travel_mcp_server.py', 'w') as f:
    f.write(server_code)

print("MCP Server created: travel_mcp_server.py")
print("Tools available: search_flights, check_hotel_availability, convert_currency, get_server_info")
print("\nNote: Exported _search_flights, _check_hotel_availability, _convert_currency functions")
print("      return JSON-serializable dicts for use with Azure AI Agent Framework")


MCP Server created: travel_mcp_server.py
Tools available: search_flights, check_hotel_availability, convert_currency, get_server_info

Note: Exported _search_flights, _check_hotel_availability, _convert_currency functions
      return JSON-serializable dicts for use with Azure AI Agent Framework


## Part 4: Testing the MCP Server Locally

Now let's test our MCP server locally. We'll:
1. Start the server in the background
2. Test individual tools
3. Verify the responses

In [5]:
# Test the tools by importing and calling the underlying functions
# Need to reload the module if it was already imported
import sys
if 'travel_mcp_server' in sys.modules:
    import importlib
    import travel_mcp_server
    importlib.reload(travel_mcp_server)
    from travel_mcp_server import _search_flights, _check_hotel_availability, _convert_currency, _get_server_info
else:
    from travel_mcp_server import _search_flights, _check_hotel_availability, _convert_currency, _get_server_info

# Test 1: Search for flights
print("=== Testing Flight Search ===")
flights = _search_flights(
    origin="New York",
    destination="London",
    departure_date="2025-11-15",
    passengers=2
)

for flight in flights:
    print(f"\n{flight['airline']} {flight['flight_number']}")
    print(f"  {flight['departure']} → {flight['arrival']}")
    print(f"  Departs: {flight['departure_time']}")
    print(f"  Arrives: {flight['arrival_time']}")
    print(f"  Price: ${flight['price']} ({flight['available_seats']} seats available)")


=== Testing Flight Search ===

Delta DL340
  New York → London
  Departs: 2025-11-15 14:15
  Arrives: 2025-11-16 02:15
  Price: $306.45 (13 seats available)

Delta UA262
  New York → London
  Departs: 2025-11-15 18:30
  Arrives: 2025-11-16 04:30
  Price: $453.5 (13 seats available)

United BA561
  New York → London
  Departs: 2025-11-15 10:45
  Arrives: 2025-11-15 22:45
  Price: $815.12 (16 seats available)


In [6]:
# Test 2: Check hotel availability
print("\n\n=== Testing Hotel Search ===")
hotels = _check_hotel_availability(
    location="London",
    check_in="2025-11-15",
    check_out="2025-11-18",
    guests=2
)

for hotel in hotels:
    print(f"\n{hotel['hotel_name']}")
    print(f"  Location: {hotel['location']}")
    print(f"  Room Type: {hotel['room_type']}")
    print(f"  Price: ${hotel['price_per_night']}/night ({hotel['available_rooms']} rooms available)")
    print(f"  Amenities: {', '.join(hotel['amenities'])}")




=== Testing Hotel Search ===

Downtown Suites
  Location: London
  Room Type: Executive Suite
  Price: $206.49/night (4 rooms available)
  Amenities: Restaurant, Pool, Parking

Seaside Resort
  Location: London
  Room Type: Suite
  Price: $338.47/night (7 rooms available)
  Amenities: Gym, Parking, Pool

Airport Inn
  Location: London
  Room Type: Standard Room
  Price: $286.14/night (3 rooms available)
  Amenities: Pool, Restaurant, WiFi

Grand Plaza Hotel
  Location: London
  Room Type: Standard Room
  Price: $324.44/night (3 rooms available)
  Amenities: WiFi, Spa, Pool


In [7]:
# Test 3: Convert currency
print("\n\n=== Testing Currency Conversion ===")
conversion = _convert_currency(
    amount=1000.0,
    from_currency="USD",
    to_currency="EUR"
)

print(f"\n{conversion['original_amount']} {conversion['original_currency']} = "
      f"{conversion['converted_amount']} {conversion['converted_currency']}")
print(f"Exchange Rate: {conversion['exchange_rate']}")
print(f"Timestamp: {conversion['timestamp']}")




=== Testing Currency Conversion ===

1000.0 USD = 850.0 EUR
Exchange Rate: 0.85
Timestamp: 2025-12-02T09:40:44.493888


## Part 5: Integrating MCP Server with Azure AI Agent

Now let's build an Azure AI Agent that can use our MCP server tools. We'll use the Azure AI Agent Framework that we've been using throughout this tutorial series.

In [8]:
# Import Azure AI Agent Framework
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.identity import DefaultAzureCredential

print("Azure AI Agent Framework imported successfully")

Azure AI Agent Framework imported successfully


In [9]:
# Create Azure AI Agent with MCP tools
from azure.identity.aio import AzureCliCredential

# Verify environment variables are set
project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model_deployment = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME")

if not project_endpoint or not model_deployment:
    print("WARNING: Missing required environment variables:")
    print("   Please set in your .env file:")
    print("   - AZURE_AI_PROJECT_ENDPOINT")
    print("   - AZURE_AI_MODEL_DEPLOYMENT_NAME")
    print("\n   See Tutorial 01b for Azure AI Foundry setup")
else:
    print("Azure AI configuration loaded")
    print(f"  Endpoint: {project_endpoint}")
    print(f"  Model: {model_deployment}")
    print("\nAgent will be created with MCP tools:")
    print("  - search_flights")
    print("  - check_hotel_availability") 
    print("  - convert_currency")


Azure AI configuration loaded
  Endpoint: https://gk-agent-framework-project.services.ai.azure.com/api/projects/agentframworkProject
  Model: gpt-4.1

Agent will be created with MCP tools:
  - search_flights
  - check_hotel_availability
  - convert_currency


In [10]:
# Test the agent with a simple query
import asyncio

async def test_agent_simple():
    """Test the agent with a simple travel query"""
    print("=" * 70)
    print("TEST 1: Simple Flight Search (Local Tools)")
    print("=" * 70)
    
    user_query = "I need to find flights from New York to London on November 15th for 2 passengers."
    
    print(f"\nUser: {user_query}\n")
    
    # Create and use the agent with Azure AI
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential,
            # Reads AZURE_AI_PROJECT_ENDPOINT and AZURE_AI_MODEL_DEPLOYMENT_NAME from env
        ) as chat_client,
    ):
        # Create agent with local Python function tools
        agent = chat_client.create_agent(
            name="TravelMCPAgent",
            instructions="""
            You are a helpful travel assistant with access to real-time travel booking tools.
            
            You can:
            - Search for flights between cities
            - Check hotel availability and pricing
            - Convert currencies for budget planning
            
            When helping users plan trips:
            1. Use the available tools to get accurate, real-time information
            2. Provide clear, organized recommendations
            3. Help with currency conversions when discussing costs
            
            Be friendly, professional, and detail-oriented.
            """,
            tools=[_search_flights, _check_hotel_availability, _convert_currency]
        )
        
        # Run the agent
        response = await agent.run(user_query)
        
        print(f"Agent: {response.text}\n")
    
    print("=" * 70)
    print("Note: This uses LOCAL Python functions as tools")
    print("      See Tutorial 16 for deploying as a remote MCP server")

# Run the test only if environment is configured
if project_endpoint and model_deployment:
    await test_agent_simple()
else:
    print("WARNING: Please configure Azure AI environment variables first")
    print("   See cell above for required configuration")

TEST 1: Simple Flight Search (Local Tools)

User: I need to find flights from New York to London on November 15th for 2 passengers.

Agent: Here are some flight options from New York to London on November 15th for 2 passengers:

1. American Airlines (Flight AA602)
   - Departure: 07:30 from New York
   - Arrival: 18:30 in London
   - Price: $914.90 per passenger
   - Available seats: 10

2. Delta (Flight UA390)
   - Departure: 11:00 from New York
   - Arrival: 18:00 in London
   - Price: $1,171.48 per passenger
   - Available seats: 35

3. United Airlines (Flight BA253)
   - Departure: 16:00 from New York
   - Arrival: 04:00 (next day) in London
   - Price: $1,066.13 per passenger
   - Available seats: 34

If you would like more details on any flight or want to proceed with a booking, please let me know your preferences!

Agent: Here are some flight options from New York to London on November 15th for 2 passengers:

1. American Airlines (Flight AA602)
   - Departure: 07:30 from New Yor

In [11]:
# Test the agent with a complex multi-tool query
async def test_agent_complex():
    """Test the agent with a complex query requiring multiple tools"""
    print("\n" + "=" * 70)
    print("TEST 2: Complex Multi-Tool Query (Local Tools)")
    print("=" * 70)
    
    user_query = """
    I'm planning a trip to London from New York. Can you help me with:
    1. Flights for November 15th for 2 passengers
    2. Hotels for 3 nights (Nov 15-18)
    3. Total cost estimate in Euros if my budget is $2000 USD
    """
    
    print(f"\nUser: {user_query}\n")
    
    # Create and use the agent with Azure AI
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential,
        ) as chat_client,
    ):
        # Create agent with local Python function tools
        agent = chat_client.create_agent(
            name="TravelMCPAgent",
            instructions="""
            You are a helpful travel assistant with access to real-time travel booking tools.
            Use multiple tools as needed to provide comprehensive travel planning assistance.
            """,
            tools=[_search_flights, _check_hotel_availability, _convert_currency]
        )
        
        # Run the agent - it will automatically use multiple tools as needed
        response = await agent.run(user_query)
        
        print(f"Agent: {response.text}\n")
    
    print("=" * 70)
    print("Note: This uses LOCAL Python functions as tools")
    print("      Tools run in the same process as the agent")

# Run the complex test only if environment is configured
if project_endpoint and model_deployment:
    await test_agent_complex()
else:
    print("WARNING: Please configure Azure AI environment variables first")


TEST 2: Complex Multi-Tool Query (Local Tools)

User: 
    I'm planning a trip to London from New York. Can you help me with:
    1. Flights for November 15th for 2 passengers
    2. Hotels for 3 nights (Nov 15-18)
    3. Total cost estimate in Euros if my budget is $2000 USD
    

Agent: Here’s a summary for your London trip from New York:

1. Flights (November 15th, 2 Passengers)
- Cheapest option: British Airways, Flight UA110
  - Departure: 09:00, Arrival: 19:00
  - Price: $304.32 per passenger
  - Total for 2: $608.64 USD

2. Hotels (3 nights, Nov 15–18, for 2 guests)
- Cheapest option: Seaside Resort
  - Suite, Price: $81.06 per night
  - Total for 3 nights: $243.18 USD

3. Total Cost Estimate in USD & Euros
- Flight + Hotel = $608.64 + $243.18 = $851.82 USD
- Your budget ($2000 USD) is converted to €1,700.00 EUR (exchange rate: 0.85)
- Estimated trip cost in Euros: €724.05

Your selections are well within budget! Would you like to book these options or see more choices?

Agent:

## Part 5b: Testing the Local MCP Server with MCPStreamableHTTPTool

Now let's test our MCP server properly using the Agent Framework's `MCPStreamableHTTPTool`. This connects to the HTTP-based MCP server running locally.

**Two MCP Tool Types:**

| Tool | Use Case | Connection |
|------|----------|------------|
| `MCPStreamableHTTPTool` | Local testing & development | Runs in YOUR process, connects to local server |
| `HostedMCPTool` | Production with Azure AI Agent Service | Azure service connects to PUBLIC URL |

**Why MCPStreamableHTTPTool for local testing:**
- Connects directly from your notebook to the local MCP server
- Tool execution happens in your process (not Azure's)
- Works with `localhost` because your code makes the HTTP calls

```
┌─────────────────┐     HTTP/SSE      ┌─────────────────────────┐
│  Your Notebook  │ ◄───────────────► │  Local MCP Server       │
│  (ChatAgent)    │   (localhost)     │  (uvicorn on port 8765) │
└─────────────────┘                   └─────────────────────────┘
```

In [17]:
# Start the MCP server locally and test it with HTTP requests
# This verifies the server works before deploying to Azure

import subprocess
import time
import requests
import os

# Start the MCP server as a background process
MCP_SERVER_PORT = 8765
MCP_SERVER_URL = f"http://localhost:{MCP_SERVER_PORT}"

print(f"Starting MCP server on {MCP_SERVER_URL}...")

# Run uvicorn in background - it will serve the ASGI app from travel_mcp_server.py
server_process = subprocess.Popen(
    ["uvicorn", "travel_mcp_server:app", "--host", "0.0.0.0", "--port", str(MCP_SERVER_PORT)],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    cwd=os.getcwd()
)

# Wait for server to start
time.sleep(3)

# Check if server is running
if server_process.poll() is None:
    print(f"✅ MCP Server started successfully (PID: {server_process.pid})")
    print(f"   URL: {MCP_SERVER_URL}")
    print(f"   SSE Endpoint: {MCP_SERVER_URL}/sse")
    
    # Test the MCP server endpoints
    print("\n--- Testing MCP Server Endpoints ---")
    
    # Test the SSE endpoint (should return 200 for GET, starts SSE stream)
    try:
        # Use a short timeout since SSE will stream indefinitely
        response = requests.get(f"{MCP_SERVER_URL}/sse", timeout=2, stream=True)
        print(f"✅ SSE endpoint (/sse): {response.status_code} - Ready for MCP connections")
        response.close()
    except requests.exceptions.ReadTimeout:
        print(f"✅ SSE endpoint (/sse): Connected (streaming mode)")
    except Exception as e:
        print(f"❌ SSE endpoint test: {e}")
    
    # Test the MCP message endpoint (POST)
    try:
        response = requests.post(f"{MCP_SERVER_URL}/mcp/", json={}, timeout=5)
        print(f"✅ MCP endpoint (/mcp/): {response.status_code}")
    except Exception as e:
        print(f"⚠️  MCP endpoint: {type(e).__name__} (expected - requires valid MCP message)")
    
    print(f"\n📡 Server is running and ready for connections!")
    print(f"   To connect with an MCP client, use: {MCP_SERVER_URL}/sse")
else:
    stdout, stderr = server_process.communicate()
    print(f"❌ Server failed to start:")
    print(f"   stdout: {stdout.decode()}")
    print(f"   stderr: {stderr.decode()}")

Starting MCP server on http://localhost:8765...
✅ MCP Server started successfully (PID: 74238)
   URL: http://localhost:8765
   SSE Endpoint: http://localhost:8765/sse

--- Testing MCP Server Endpoints ---
✅ SSE endpoint (/sse): 404 - Ready for MCP connections
✅ MCP endpoint (/mcp/): 406

📡 Server is running and ready for connections!
   To connect with an MCP client, use: http://localhost:8765/sse
✅ MCP Server started successfully (PID: 74238)
   URL: http://localhost:8765
   SSE Endpoint: http://localhost:8765/sse

--- Testing MCP Server Endpoints ---
✅ SSE endpoint (/sse): 404 - Ready for MCP connections
✅ MCP endpoint (/mcp/): 406

📡 Server is running and ready for connections!
   To connect with an MCP client, use: http://localhost:8765/sse


In [18]:
# Test the local MCP server using MCPStreamableHTTPTool
# This is the proper way to test local MCP servers with Agent Framework

from agent_framework import MCPStreamableHTTPTool, ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

async def test_local_mcp_server():
    """Test agent with local MCP server using MCPStreamableHTTPTool"""
    print("=" * 70)
    print("Testing Local MCP Server with MCPStreamableHTTPTool")
    print("=" * 70)
    
    # Create MCP tool pointing to our local server
    mcp_tool = MCPStreamableHTTPTool(
        name="Travel Booking MCP",
        url=f"{MCP_SERVER_URL}/mcp/",  # FastMCP uses /mcp/ endpoint
        description="Travel booking tools via local MCP server"
    )
    
    print(f"Connecting to MCP server at: {MCP_SERVER_URL}/mcp/")
    
    async with (
        AzureCliCredential() as credential,
        mcp_tool,  # Connect to MCP server
        ChatAgent(
            chat_client=AzureAIAgentClient(async_credential=credential),
            name="TravelAgent",
            instructions="You are a travel assistant. Use the MCP tools to help with travel planning.",
            tools=mcp_tool,  # Pass MCP tool to agent
        ) as agent,
    ):
        # Show available tools from MCP server
        print(f"\n📋 MCP Server Tools Loaded: {len(mcp_tool.functions)}")
        for func in mcp_tool.functions:
            print(f"   - {func.name}: {func.description[:50] if func.description else 'No description'}...")
        
        # Test query
        query = "Search for flights from New York to London on November 20th"
        print(f"\n🧑 User: {query}")
        
        response = await agent.run(query)
        print(f"\n🤖 Agent: {response.text}")
    
    print("\n" + "=" * 70)
    print("✅ Successfully tested local MCP server!")
    print("=" * 70)

# Run test if environment is configured and server is running
if project_endpoint and model_deployment and server_process and server_process.poll() is None:
    await test_local_mcp_server()
else:
    if not (project_endpoint and model_deployment):
        print("⚠️  Azure AI not configured - see Part 5 for setup")
    if not server_process or server_process.poll() is not None:
        print("⚠️  MCP server not running - run the previous cell first")

Testing Local MCP Server with MCPStreamableHTTPTool
Connecting to MCP server at: http://localhost:8765/mcp/

📋 MCP Server Tools Loaded: 4
   - search_flights: Search for available flights between two cities.

...
   - check_hotel_availability: Check hotel room availability in a specific locati...
   - convert_currency: Convert amount from one currency to another.

Args...
   - get_server_info: Get information about this MCP server...

🧑 User: Search for flights from New York to London on November 20th

📋 MCP Server Tools Loaded: 4
   - search_flights: Search for available flights between two cities.

...
   - check_hotel_availability: Check hotel room availability in a specific locati...
   - convert_currency: Convert amount from one currency to another.

Args...
   - get_server_info: Get information about this MCP server...

🧑 User: Search for flights from New York to London on November 20th

🤖 Agent: Here are some available flights from New York to London on November 20th:

1. Delta 

In [20]:
# Clean up: Stop the MCP server when done
print("Stopping MCP server...")

if server_process and server_process.poll() is None:
    server_process.terminate()
    server_process.wait(timeout=5)
    print(f"✅ MCP Server stopped (PID: {server_process.pid})")
else:
    print("Server was not running")

# Note: In production, the MCP server would run continuously
# and be deployed to Azure Container Apps (see Tutorial 16)

Stopping MCP server...
✅ MCP Server stopped (PID: 74238)


In [19]:
# HostedMCPTool Pattern - For Production (Documentation Only)
# NOTE: HostedMCPTool requires Azure AI Agent Service to reach the MCP server
#       This means the server must be deployed to a PUBLIC URL (not localhost)

from agent_framework import HostedMCPTool

print("=" * 70)
print("HostedMCPTool Pattern (for Azure AI Agent Service)")
print("=" * 70)

print("""
When deploying to production with Azure AI Agent Service:

1. Deploy your MCP server to Azure Container Apps (Tutorial 16)
2. Use HostedMCPTool with the public URL:

```python
from agent_framework import HostedMCPTool

# HostedMCPTool - Azure AI Agent Service makes the HTTP calls
mcp_tool = HostedMCPTool(
    name="Travel Booking MCP",
    url="https://your-mcp.azurecontainerapps.io/mcp/",
    approval_mode="never_require"
)

# Agent created with Azure AI Agent Service
agent = chat_client.create_agent(
    name="TravelAgent",
    tools=mcp_tool  # Azure service will call this URL
)
```

Key difference:
- MCPStreamableHTTPTool: YOUR code makes HTTP calls (works with localhost)
- HostedMCPTool: AZURE makes HTTP calls (needs public URL)
""")

print("=" * 70)

HostedMCPTool Pattern (for Azure AI Agent Service)

When deploying to production with Azure AI Agent Service:

1. Deploy your MCP server to Azure Container Apps (Tutorial 16)
2. Use HostedMCPTool with the public URL:

```python
from agent_framework import HostedMCPTool

# HostedMCPTool - Azure AI Agent Service makes the HTTP calls
mcp_tool = HostedMCPTool(
    name="Travel Booking MCP",
    url="https://your-mcp.azurecontainerapps.io/mcp/",
    approval_mode="never_require"
)

# Agent created with Azure AI Agent Service
agent = chat_client.create_agent(
    name="TravelAgent",
    tools=mcp_tool  # Azure service will call this URL
)
```

Key difference:
- MCPStreamableHTTPTool: YOUR code makes HTTP calls (works with localhost)
- HostedMCPTool: AZURE makes HTTP calls (needs public URL)



## Understanding the Three MCP Tool Patterns

### Pattern 1: Local Python Functions (Part 5)
```python
# Tools are Python functions in the same process
agent = chat_client.create_agent(
    tools=[_search_flights, _check_hotel_availability, _convert_currency]
)
```
✅ **Best for:** Development, prototyping, single-agent apps

### Pattern 2: MCPStreamableHTTPTool (Part 5b)
```python
# Connect to local MCP server via HTTP from YOUR code
mcp_tool = MCPStreamableHTTPTool(
    name="Travel Booking MCP",
    url="http://localhost:8765/mcp/"
)
async with mcp_tool:
    agent = ChatAgent(tools=mcp_tool)
```
✅ **Best for:** Testing MCP servers locally before deployment

### Pattern 3: HostedMCPTool (Tutorial 16 - Production)
```python
# Azure AI Agent Service connects to PUBLIC MCP server
mcp_tool = HostedMCPTool(
    name="Travel Booking MCP",
    url="https://your-mcp.azurecontainerapps.io/mcp/",
    approval_mode="never_require"
)
agent = chat_client.create_agent(tools=mcp_tool)
```
✅ **Best for:** Production, multi-agent systems, enterprise

### Comparison Table

| Pattern | Who makes HTTP calls? | Works with localhost? | Use Case |
|---------|----------------------|----------------------|----------|
| Local Functions | N/A (in-process) | N/A | Development |
| MCPStreamableHTTPTool | Your notebook/code | ✅ Yes | Local testing |
| HostedMCPTool | Azure AI Agent Service | ❌ No (needs public URL) | Production |

## Part 6: Best Practices for MCP Server Development

### Critical Lessons Learned

**1. Dual Function Pattern (Required for Agent Integration)**
```python
# Underscore functions return JSON-serializable dicts (for agents)
def _search_flights(...) -> List[Dict]:
    return [{"flight_number": "AA123", ...}]

# MCP wrappers convert to Pydantic models (for MCP server)
@mcp.tool()
def search_flights(...) -> List[Flight]:
    return [Flight(**f) for f in _search_flights(...)]
```

**2. Package Management**
- Don't pin FastMCP versions - use `pip install fastmcp` without constraints
- Let pip auto-resolve dependencies (FastMCP evolves rapidly)

**3. Environment & Security**
- Store credentials in Azure Keyvault
- Validate environment variables before agent creation
- Never hardcode sensitive values

**4. Testing & Error Handling**
- Test underscore functions locally first
- Test with agents to verify JSON serialization
- Wrap tool functions in try-except blocks
- Return structured error responses

**5. Production Considerations**
- Make tools async for I/O operations
- Implement rate limiting and timeouts
- Use caching for frequently requested data
- Monitor tool invocations and error rates

## Summary and Key Takeaways

### What You've Learned

1. **Model Context Protocol (MCP)**: A standardized way to connect AI agents to external tools and data sources

2. **FastMCP Framework**: Python framework for building MCP servers with:
   - Decorator-based tool definitions
   - Automatic schema generation
   - Built-in validation

3. **Tool Development**: How to create MCP tools with:
   - Clear function signatures
   - Pydantic models for structured data
   - Proper documentation

4. **Agent Integration**: How to connect MCP servers to AI agents using:
   - OpenAI function calling
   - Tool execution handlers
   - Multi-turn conversations

### Next Steps

In the next tutorial (**16_deploy_fastmcp_to_aca.ipynb**), you'll learn how to:
- Dockerize your MCP server
- Deploy to Azure Container Apps
- Configure environment variables and scaling
- Monitor production MCP servers

### Additional Resources

- **FastMCP Documentation**: https://github.com/jlowin/fastmcp
- **MCP Specification**: https://modelcontextprotocol.io
- **Azure Container Apps**: https://learn.microsoft.com/azure/container-apps/

---

**Congratulations!** You've built your first MCP server and integrated it with an AI agent. This is the foundation for building scalable, enterprise-grade AI agent architectures.